# Day 14 
## Part 1
Roll by splitting a column by immobile rocks and for each section putting the rolling rocks at the start.

In [6]:
def parse_data(s):
    return s.strip().splitlines()

def roll_north(column):
    sections = column.split("#")
    return "#".join(
        [
            section.count("O") * "O" + section.count(".") * "."
            for section in sections
        ]
    )

def north_load(column):
    return sum(
        l 
        for c, l in zip(column, range(len(column), 0, -1))
        if c == "O"
    )

def part_1(data):
    return sum(
        north_load(roll_north("".join(column)))
        for column in zip(*data)
    )

test_data = parse_data("""O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....""")

assert part_1(test_data) == 136

In [8]:
data = parse_data(open("input").read())

part_1(data)

103333

## Part 2
Find a cycle within the cycles by looking for a repeated map and doing modular arithmetic.

First define the different rolls and cycle.

In [34]:
roll = roll_north

def spin_north(data):
    new_cols = [
        roll("".join(column))
        for column in zip(*data)
    ]
    return ["".join(r) for r in zip(*new_cols)]

n = spin_north(test_data)
n

['OOOO.#.O..',
 'OO..#....#',
 'OO..O##..O',
 'O..#.OO...',
 '........#.',
 '..#....#.#',
 '..O..#.O.O',
 '..O.......',
 '#....###..',
 '#....#....']

In [25]:
def sreversed(xs):
    return "".join(list(reversed(xs)))

def spin_east(data):
    new_rows = [
        sreversed(roll(sreversed(column)))
        for column in data
    ]
    return new_rows

e = spin_east(n)
e

['.OOOO#...O',
 '..OO#....#',
 '..OOO##..O',
 '..O#....OO',
 '........#.',
 '..#....#.#',
 '....O#..OO',
 '.........O',
 '#....###..',
 '#....#....']

In [26]:
def spin_south(data):
    new_cols = [
        sreversed(roll(sreversed(column)))
        for column in zip(*data)
    ]
    return ["".join(r) for r in zip(*new_cols)]

s = spin_south(e)
s

['...OO#...O',
 '..OO#....#',
 '..OO.##...',
 '..O#....OO',
 '..O.....#O',
 '..#....#.#',
 '.....#....',
 '..........',
 '#...O###.O',
 '#O..O#..OO']

In [28]:
def spin_west(data):
    new_rows = [
        roll(column)
        for column in data
    ]
    return new_rows

w = spin_west(s)
w

['OO...#O...',
 'OO..#....#',
 'OO...##...',
 'O..#OO....',
 'O.......#O',
 '..#....#.#',
 '.....#....',
 '..........',
 '#O...###O.',
 '#OO..#OO..']

Oops, going clockwise there, should be anti-clockwise.

In [33]:
def compose(*fs):
    def composed(x):
        for f in fs:
            x = f(x)
        return x
    return composed

cycle = compose(spin_north, spin_west, spin_south, spin_east)

cycle(test_data)

['.....#....',
 '....#...O#',
 '...OO##...',
 '.OO#......',
 '.....OOO#.',
 '.O#...O#.#',
 '....O#....',
 '......OOOO',
 '#...O###..',
 '#..OO#....']

Find out when a cycle repeats.

In [53]:
def find_cycle(data):
    n = 0
    seen = {tuple(data): 0}

    while True:
        data = cycle(data)
        n += 1
        t = tuple(data)
        if t in seen:
            return seen[t], n - seen[t]
        seen[t] = n
        
find_cycle(test_data)

(3, 7)

Use the cycle start and cycle length to find the number of full spins to get to the same map as after the big number. Inefficiently repeat those spins to get the map.

In [66]:
def load(data):
    total = 0
    for n, line in zip(range(len(data), 0, -1), data):
        total += line.count("O") * n
    return total

def part_2(data):
    cycle_start, cycle_length = find_cycle(data)
    i = cycle_start + ((1000000000 - cycle_start) % cycle_length)
    print(cycle_start, cycle_length, i)
    for n in range(i):
        data = cycle(data)
    return load(data)

assert part_2(test_data) == 64

3 7 6


In [68]:
%%time

part_2(data)

99 36 100
CPU times: user 1.12 s, sys: 5.77 ms, total: 1.12 s
Wall time: 1.12 s


97241

I've gone for ease of coding over efficiency in this one.